In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.1f}'.format

In [2]:
def read_table(name):
    file_root = "https://storage.googleapis.com/genx_2018/"
    return pd.read_table(file_root + name, sep="\t")

#phenotype_meta_table = read_table("Melanoma_Phenotype_Metadata.txt")

### Mutations

In [45]:
mutation = read_table("Melanoma_Mutation.txt")
mutation.head()

,Sample,Chr,Start,Stop,Ref,Alt,Gene,Effect,DNA_VAF,RNA_VAF,Amino_Acid_Change
0,TCGA-D3-A3ML-06,chr5,140182973,140182973,G,A,PCDHA3,Missense_Mutation,0.5,nan,p.D731N
1,TCGA-D3-A3ML-06,chr2,133541884,133541884,C,T,NCKAP5,Missense_Mutation,0.5,nan,p.E834K
2,TCGA-D3-A3ML-06,chr19,51217544,51217544,C,T,SHANK1,Missense_Mutation,0.2,nan,p.G179R
3,TCGA-D3-A3ML-06,chr8,2820757,2820757,G,A,CSMD1,Silent,0.5,nan,p.I3148I
4,TCGA-D3-A3ML-06,chr19,43708353,43708353,C,T,PSG4,Missense_Mutation,0.5,nan,p.E39K


In [46]:
# get rid of nans in start stop gene
mutation = mutation[np.isfinite(mutation.Start)]
mutation = mutation[np.isfinite(mutation.Stop)]

# create Start-Stop-Gene identifier
temp_df = mutation[['Start', 'Stop', 'Gene', 'Chr']]
temp_df.Start = temp_df.Start.astype(str)
temp_df.Stop = temp_df.Stop.astype(str)
mutation['m_key'] = temp_df.apply(lambda x: ''.join(x), axis=1)

/home/nela/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [47]:
# check uniqueness
mutation.shape[0], mutation.m_key.nunique()

(422553, 361488)

In [48]:
whynotunique = mutation.groupby('m_key').count()
whynotunique[whynotunique.Sample > 100]

,Sample,Chr,Start,Stop,Ref,Alt,Gene,Effect,DNA_VAF,RNA_VAF,Amino_Acid_Change
m_key,,,,,,,,,,,
140453136140453136BRAFchr7,210,210,210,210,210,210,210,210,210,0,210


whatever.

In [49]:
mutation.head(1)

,Sample,Chr,Start,Stop,Ref,Alt,Gene,Effect,DNA_VAF,RNA_VAF,Amino_Acid_Change,m_key
0,TCGA-D3-A3ML-06,chr5,140182973,140182973,G,A,PCDHA3,Missense_Mutation,0.5,nan,p.D731N,140182973140182973PCDHA3chr5


In [53]:
# are they all really useless?
cols_to_drop = ['Chr', 'Start', 'Stop', 'Gene', 'DNA_VAF', 'RNA_VAF', 'Amino_Acid_Change']
mutation = mutation.drop(cols_to_drop, axis='columns')
mutation['Mutated'] = mutation[['Ref', 'Alt']].apply(lambda x: 1 if x.Ref != x.Alt else 0)

ValueError: labels ['Chr' 'Start' 'Stop' 'Gene' 'DNA_VAF' 'RNA_VAF' 'Amino_Acid_Change'] not contained in axis

In [43]:
mutation.groupby(['m_key', 'Sample']).mean()

DataError: No numeric types to aggregate

### Methylations

In [6]:
methylationr = read_table("Melanoma_Methylation.txt")

In [7]:
methylation = methylationr.head(500)

In [8]:
# get rid of nans in start stop gene
methylation = methylation[np.isfinite(methylation.Start)]
methylation = methylation[np.isfinite(methylation.Stop)]

# to be able to get integers from floats
methylation.Start = methylation.Start.astype(int)
methylation.Stop = methylation.Stop.astype(int)

# to get the same identifier as for the mutations
temp_df = methylation[['Start', 'Stop', 'Gene']]
temp_df.Start = temp_df.Start.astype(str)
temp_df.Stop = temp_df.Stop.astype(str)
temp_df.Gene = temp_df.Gene.astype(str)
methylation['m_key'] = temp_df.apply(lambda x: ''.join(x), axis=1)

/home/nela/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
# check if the constructed identifier is unique
methylation.shape[0] == methylation.m_key.nunique()

True

In [10]:
cols_to_drop = ['probeID', 'Chr', 'Start', 'Stop', 'Gene', 'Relation_CpG_Island']
methylation = methylation.drop(cols_to_drop, axis='columns')

In [11]:
# separate by Strand
methylation_minus = methylation[methylation.Strand == '-'].drop('Strand', axis=1).T
methylation_plus = methylation[methylation.Strand == '+'].drop('Strand', axis=1).T

In [12]:
# move join key from index to col 'Sample
methylation_plus = methylation_plus.reset_index()
methylation_plus = methylation_plus.rename(columns={'index':'Sample'})

In [33]:
pd.merge(mutation, methylation_plus, on='Sample', how='inner').head(2)

,Sample,Chr,Start,Stop,Ref,Alt,Gene,Effect,DNA_VAF,RNA_VAF,Amino_Acid_Change,m_key,3,4,7,10,12,16,17,22,25,26,27,28,29,30,31,32,33,36,41,42,43,44,46,47,55,57,60,61,64,65,67,68,69,71,74,76,77,78,...,412,413,414,415,417,418,419,420,421,424,425,426,430,431,432,434,435,436,437,440,441,443,446,449,452,456,458,461,462,463,464,466,467,468,470,471,473,474,475,476,479,480,482,484,486,489,490,492,493,498
0,TCGA-D3-A3ML-06,chr5,140182973,140182973,G,A,PCDHA3,Missense_Mutation,0.5,nan,p.D731N,140182973140182973PCDHA3chr5,0.9,0.9,0.8,0.8,1.0,NaN,0.1,0.0,NaN,0.9,0.9,0.9,0.5,NaN,0.1,0.7,NaN,1.0,0.9,NaN,NaN,0.9,0.0,0.8,NaN,0.9,0.8,0.7,0.9,0.9,1.0,0.1,0.9,NaN,0.9,0.0,0.8,0.1,...,0.6,1.0,NaN,0.0,0.5,1.0,NaN,NaN,NaN,0.9,0.9,0.4,0.8,0.0,0.9,NaN,0.0,0.9,NaN,0.5,0.9,0.5,1.0,0.7,0.7,NaN,NaN,0.0,0.9,NaN,NaN,0.9,0.9,0.1,0.1,0.9,1.0,0.1,0.9,0.9,0.8,NaN,0.1,0.9,0.9,1.0,0.8,1.0,NaN,0.1
1,TCGA-D3-A3ML-06,chr2,133541884,133541884,C,T,NCKAP5,Missense_Mutation,0.5,nan,p.E834K,133541884133541884NCKAP5chr2,0.9,0.9,0.8,0.8,1.0,NaN,0.1,0.0,NaN,0.9,0.9,0.9,0.5,NaN,0.1,0.7,NaN,1.0,0.9,NaN,NaN,0.9,0.0,0.8,NaN,0.9,0.8,0.7,0.9,0.9,1.0,0.1,0.9,NaN,0.9,0.0,0.8,0.1,...,0.6,1.0,NaN,0.0,0.5,1.0,NaN,NaN,NaN,0.9,0.9,0.4,0.8,0.0,0.9,NaN,0.0,0.9,NaN,0.5,0.9,0.5,1.0,0.7,0.7,NaN,NaN,0.0,0.9,NaN,NaN,0.9,0.9,0.1,0.1,0.9,1.0,0.1,0.9,0.9,0.8,NaN,0.1,0.9,0.9,1.0,0.8,1.0,NaN,0.1


In [32]:
methylation_plus.head(2)

,Sample,3,4,7,10,12,16,17,22,25,26,27,28,29,30,31,32,33,36,41,42,43,44,46,47,55,57,60,61,64,65,67,68,69,71,74,76,77,78,79,80,82,83,84,86,87,88,89,90,92,...,412,413,414,415,417,418,419,420,421,424,425,426,430,431,432,434,435,436,437,440,441,443,446,449,452,456,458,461,462,463,464,466,467,468,470,471,473,474,475,476,479,480,482,484,486,489,490,492,493,498
0,TCGA-FR-A728-01,0.5,0.8,0.5,0.8,0.9,NaN,0.1,0.0,NaN,0.8,0.9,0.9,0.5,NaN,0.0,0.5,NaN,0.2,0.7,NaN,NaN,0.9,0.0,0.7,NaN,0.9,0.8,0.9,0.7,0.9,0.9,0.1,0.5,NaN,0.7,0.0,0.4,0.0,0.4,0.6,0.7,0.6,0.8,0.0,NaN,NaN,NaN,NaN,0.4,...,0.2,0.9,NaN,0.0,0.2,0.9,NaN,NaN,NaN,0.5,0.9,0.2,0.8,0.4,0.4,NaN,0.0,0.7,NaN,0.6,0.9,0.4,0.9,0.5,0.4,NaN,NaN,0.1,0.8,NaN,NaN,0.9,0.4,0.1,0.1,0.4,1.0,0.3,0.9,0.5,0.7,NaN,0.6,0.7,0.8,1.0,0.4,0.9,NaN,0.0
1,TCGA-GN-A4U8-11,0.4,0.9,0.4,0.9,0.9,NaN,0.1,0.0,NaN,0.6,0.8,0.8,0.5,NaN,0.0,0.0,NaN,0.1,0.7,NaN,NaN,0.7,0.0,0.5,NaN,0.9,0.8,0.9,0.9,0.9,1.0,0.1,0.4,NaN,0.9,0.0,0.2,0.0,0.4,0.7,0.9,0.6,0.8,0.0,NaN,NaN,NaN,NaN,0.2,...,0.2,0.9,NaN,0.0,0.0,0.9,NaN,NaN,NaN,0.3,0.8,0.3,0.7,0.5,0.5,NaN,0.0,0.8,NaN,0.6,0.8,0.1,0.9,0.8,0.3,NaN,NaN,0.0,0.8,NaN,NaN,0.9,0.6,0.1,0.1,0.3,1.0,0.5,0.9,0.4,0.7,NaN,0.8,0.7,0.8,1.0,0.6,0.9,NaN,0.0
